<h1> Decision Tree </h1>

Decision Tree é um poderoso algoritmo de aprendizado de máquina que também permite regressão e classificação. Também é capaz de trabalhar com multiplas saídas.

In [10]:
#Bibliotecas
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

#Carrega dataset iris
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target
tree_clf = DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
from sklearn.tree import export_graphviz

export_graphviz(tree_clf,out_file="iris_tree.dot",feature_names=iris.feature_names[2:],
                class_names=iris.target_names,rounded=True,filled=True)

<img src='imagens/arvoredecisao.png'> </img>

NOTA: Arvóres de decisão não precisam de feature scaling e nem centering

A pontuação de Gini mede a impureza do nó, se um nór for "puro" (gini=0) todas as instâncias pertecem a mesma classe.

Na celula verde, o gini foi calculado da seguinte maneira: $1-(0/54)^2-(49/54)^2-(5/54)≈0.168$

<img src='imagens/gini.png'> </img>

NOTA: o Scikit usa algoritmo CART, que produz apenas árvores binárias

<img src='imagens/boundaries.png'> </img>

Se o max_depth for setado para 3, então os dois nós depth-2 poderiam adicionar outra fronteira de decisão (pontilhada na imagem)

In [12]:
#Mostrando as probabilidades para cada classe
tree_clf.predict_proba([[5, 1.5]])

array([[ 0.        ,  0.90740741,  0.09259259]])

In [13]:
#Fazendo predição
tree_clf.predict([[5, 1.5]])

array([1])

<h3> Algoritmo de treinamento CART </h3>

O algoritmo _Classification_ _and_ _Regression_ _Tree_ (CART), também chamado de arvores crescentes. A ideia é bem simples, o algoritmo separa o treinamento em dois subconjuntos usando apenas uma feature k e threshold $t_k$. Para escolher o k e o $t_k$ ele procura pelo par (k,$t_k$) que produz os subconjuntos mais puros (ponderados pelo tamanho). 

A função de custo é dado pela equação 6-2:

<img src="imagens/cartcost.png"> </img>

Após separar em dois subconjuntos usando a mesma lógica, o algoritmo vai executando recursivamente, ele para quando alcança a máxima profundidade (hyperparâmetro _max_ _depth_) ou quando não encontra uma separação que possa redudizar a impuridade. Outros hyperparâmetros controla condições adicionais de parada (min_samples_split, min_samples_leaf, min_weight_fraction_leaf, and max_leaf_nodes).

ATENÇÃO: CART é um algoritmo guloso, ele procura pela separação ótima apenas no level mais alto, e vai repetindo o processo abaixo, ele não testa se uma separação pode levar a um nível de impureza menor considerando os vários níveis abaixo, não garantindo que a solução seja ótima, porém produz bons resultados.

Infelizmente encontrar uma árvore ótima é um problema NP-Completo, ele tem complexidade de $O(exp(m))$, fazendo o problema ser intratável mesmo com conjuntos pequenos. Com relação a isso uma solução aproximada é aceitável.

<h3> Complexidade computacional </h3>

Para fazer as predições requer apenas atravessar a arvóre da raiz até a folha, como as árvores são aproximadamente balanceadas, a travessia requer grosseiramente $O(log_2(m))$, então a predição é muito rápida. 

Entretanto, o algoritmo de treinamento compara toda as features (ou menos se _max_features_ estiver configurado) em todas as amotras em cada nó, isso resulta em uma complexidade de $O(n × m × log(m))$. Para conjuntos de treinos pequenos (menos que alguns milhares), o Scikit pode aumentar a velocidade pre´-ordenando os dados (parâmetro _presort=True_), mas isso diminui muito o treinamento para conjuntos grandes.

<h3> Impureza de gini ou Entropia? </h3>

Por default o algoritmo usa impureza de Gini, mas é possível selecionar a medida de impureza entropia, escolhendo o parâmetro _criterion_ para "entropy". O conceito de entropia originado na termodinâmica como medida de desordem molecular: a entropia se apróxima de zero quando as moléculas estão paradas e bem ordenadas. Em aprendizado de máquina, essa medida é frequentemente usada como impureza: o valor é zero quando contém instâncias de apenas uma classe. A equação 6-3 mostra a definição de entropia para nó i.

<img src="imagens/entropia.png"> </img>

A verdade é que na maioria das vezes, não faz muita diferença usar um o outro. A impureza de Gini é um pouco mais rápida de computar, então é boa como default. Entretanto, quando se diferem, a impureza de Gini tende a isolar as classes mais frequentes em seu proprio segmento de árvore, enquanto entropia tende a produzir um pouco mais de árvores balanceadas.

<h3> Hyperparâmetro de regularização </h3>

Diferentes de outro modelos como regressão, você não faz uma assunção sobre a distribuição dos dados. Para poder regularizar um modelo como árvore de decisão existem diversos parâmetros que podem ser controlados.

_max_depth_: Número máximo de camadas que podem ser criados

_max_leaf_nodes_: Número máximo de nós de folha

_max_features_:máximo número de features que são avaliadas para separar cada nó



_min_samples_split_: o número de minímo de amostras que nó precisa ter antes de ser separado

_min_samples_leaf: o número mínimo de amostras que uma folha precisa ter

_min_weight_fraction_leaf: mesmo do anterior, porém mas expresso em fração do número total de instâncias ponderadas

__* Aumentando _min*_ ou diminuindo _max*_ irá regularizar o modelo. __


NOTA: Outros algoritmos trabalham primeiramente treinando a árvore de decisão sem restrições, e depois deletando nós desnecessários. O nó cujos filhos são todos folhas é considerado desnecessário se a melhoria de pureza que ele fornece não for estatisticamente significativa. Testes estatísticos padrão, são usados para estimar a probabilidade que a melhoria irá causar, se a probabilidade for maior que um dado threshold (tipicamente 5%, controlado por hiper), então o nó é considerado desnecessário e os filhos deletados. A "poda" da árvore continua até todos os nós serem "podados".

A figura 6-3 mostra árvore de decisão no dataset moons. Na esquerda com parâmetros default, e na direita com _min_samples_leaf_=4. Na esquerda ocorre um overfitting e da direita generaliza melhor.

<img src="imagens/regularizacao.png"> </img>

<h3> Regressão </h3>

In [14]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

A diferença que em vez de prever a classe em cada nó, ele prevê um valor. Só que agora em vez de minimizar o grau de impureza, você minimiza o MSE (Mean Squared Error). O modelo criado é mostrado na Figura 6-5, o valor predito basicamente vai ser a média das instâncias em cada região. O algoritmo separa cada região de maneira que as instâncias fiquem mais próximas possíveis do valor predito.

<img src="imagens/regressao.png"> </img>

O algoritmo CART funciona da mesma maneira como explicado anteriormente, exceto que em vez de separar para minimizar a pureza, ele minimiza o MSE. Na Equação 6-4 é possível ver a equação de custo que algoritmo tentar minimizar.

<img src="imagens/cost.png"> </img>

Como as tarefas de classificações, árvores de decisão são propensos a overfitting quando lidando com tarefas de regressão, com valores default você tem resultado da esquerda na Figura 6-6, que obviamente está ocorrendo overfitting, apenas mudando o parâmetro min_samples_leaf=10 o resultado se torna mais geral como a Figura mostrado no lado direito.

<img src="imagens/regover.png"> </img>

A conclusão é que as árvores de decisão muito boas para lidar com decisões ortogonais, fazendo elas insensiveis a rotação no conjunto de treino. Além de serem fáceis de entender, interpretar e fácil de usar. Na figura 6-7 é possível observar duas separações que engloba todo o dataset, porém ao rotacionar na Figura direita a decisão não parecer generalizar muito bem. 

Uma maneira de se lidar com esse problema é usando PCA que frequentemente resulta em uma melhor orientação do conjunto de treino.

<img src="imagens/sensivel.png"> </img>

O principal problema e que são muito sensíveis a pequenas variações no conjunto de treino. E bom ter em mente que o algoritmo do Scikit é estocástico, e você pode ter diferentes modelos mesmo com os mesmos dados de treino, a menos que o parâmetro random_state esteja configurado

Como você pode ver na Figura 6-8, quando remove a mais larga Iris do dataset as superficies de decisão do modelo se tornam completamente diferentes, se comparada ao vista anteriormente na Figura 6-2

<img src="imagens/sensivel2.png"> </img>

Random Forest consegue limitar está instabilidade calculando média ao longo de várias árvores, que será visto no próximo cápitulo.